<a href="https://colab.research.google.com/github/kumar045/Assignment-For-Filed/blob/main/YTUBE_018_PUBLIC_LLAMA3_02_FINE_TUNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FINE TUNINING USING UNSLOTH

In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/YouTube/')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import pandas as pd
from pprint import pprint

list_ds =[
{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
{"instruction":"Which company is the Acquirer.", "input":"Wonderful Products Inc, a leading company in the consumer goods industry, has successfully completed its acquisition of Noble Solutions Group, a prominent provider of innovative solutions.", "output":"Wonderful Products Inc"},

{"instruction":"Which company is the Acquirer.", "input":"ABC Corporation has completed acquisition of 91% stake in French IT Consulting Company, Rene Consulting", "output":"ABC Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"JBL Technologies has completed acquisition of 51% stake in California Based Defence Company, MaryLand", "output":"JBL Technologies"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"Which company is the Acquirer.", "input":"Summit Solutions Inc, a leading technology solutions provider, has announced its plans to acquire Accelerate Innovations Group, a prominent innovation and consulting firm.", "output":"Summit Solutions Inc"}
]

pprint(list_ds)

[{'input': 'Peak Performance Corporation to Acquire Power Forward Corporation '
           'by 28/Jul/2022',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Peak Performance Corporation'},
 {'input': 'Prime Solutions Group Acquires Dynamic Solutions Inc in a '
           'Strategic Merger',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Prime Solutions Group'},
 {'input': 'Wonderful Products Inc, a leading company in the consumer goods '
           'industry, has successfully completed its acquisition of Noble '
           'Solutions Group, a prominent provider of innovative solutions.',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Wonderful Products Inc'},
 {'input': 'ABC Corporation has completed acquisition of 91% stake in French '
           'IT Consulting Company, Rene Consulting',
  'instruction': 'Which company is the Acquirer.',
  'output': 'ABC Corporation'},
 {'input': 'JBL Technologies has completed acquisition of 51% stake

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [ ]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)



config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla V100-SXM2-16GB. Max memory: 15.773 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 7
})


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 7
})


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla V100-SXM2-16GB. Max memory = 15.773 GB.
5.668 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.342900
2,3.445600
3,3.359900
4,3.003600
5,2.665000
6,2.252100
7,1.811800
8,1.407500
9,1.062400
10,0.794200


In [ ]:
model.save_pretrained_merged("model_mna", tokenizer, save_method = "merged_16bit",)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 36.51 out of 50.99 RAM for saving.


100%|██████████| 32/32 [00:27<00:00,  1.17it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
